In [2]:
import torch
import ssl
import matplotlib.pyplot as plt
from src.logger.logger_initializer import LoggerInitializer

ssl._create_default_https_context = ssl._create_unverified_context
device = 'cuda'
torch.__version__

LoggerInitializer().init()

[2023-04-16 23:39:45] 20183 root {logger_initializer-47} INFO - Current process ID: 20183


True

In [3]:
from src.data_loader.dataset_getter import DatasetGetter
dataset = DatasetGetter('../../bayesian_test/data', batch_size=12).get()

In [38]:
import os

import numpy as np
import torch
from pyro.contrib.examples.util import MNIST
import torch.nn as nn
import torchvision.transforms as transforms

import pyro
import pyro.distributions as dist
import pyro.contrib.examples.util  # patches torchvision
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

class Decoder(nn.Module):
    image_dimension = 224*224*3

    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        # setup the two linear transformations used
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, self.image_dimension)
        # setup the non-linearities
        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()

    def forward(self, z):
        # define the forward computation on the latent z
        # first compute the hidden units
        hidden = self.softplus(self.fc1(z))
        # return the parameter for the output Bernoulli
        # each is of size batch_size x 784
        loc_img = self.sigmoid(self.fc21(hidden))
        return loc_img

class Encoder(nn.Module):
    image_dimension = 224*224*3

    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        # setup the three linear transformations used
        self.fc1 = nn.Linear(self.image_dimension, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        self.fc22 = nn.Linear(hidden_dim, z_dim)
        # setup the non-linearities
        self.softplus = nn.Softplus()

    def forward(self, x):
        # define the forward computation on the image x
        # first shape the mini-batch to have pixels in the rightmost dimension
        x = x.reshape(-1, self.image_dimension)
        # then compute the hidden units
        hidden = self.softplus(self.fc1(x))
        # then return a mean vector and a (positive) square root covariance
        # each of size batch_size x z_dim
        z_loc = self.fc21(hidden)
        z_scale = torch.exp(self.fc22(hidden))
        return z_loc, z_scale

class VAE(nn.Module):
    image_dimension = 224*224*3

    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, z_dim=50, hidden_dim=400, use_cuda=False):
        super().__init__()
        # create the encoder and decoder networks
        self.encoder = Encoder(z_dim, hidden_dim)
        self.decoder = Decoder(z_dim, hidden_dim)

        if use_cuda:
            # calling cuda() here will put all the parameters of
            # the encoder and decoder networks into gpu memory
            self.cuda()
        self.use_cuda = use_cuda
        self.z_dim = z_dim

    # define the model p(x|z)p(z)
    def model(self, x):
        # register PyTorch module `decoder` with Pyro
        pyro.module("decoder", self.decoder)
        with pyro.plate("data", x.shape[0]):
            # setup hyperparameters for prior p(z)
            z_loc = x.new_zeros(torch.Size((x.shape[0], self.z_dim)))
            z_scale = x.new_ones(torch.Size((x.shape[0], self.z_dim)))
            # sample from prior (value will be sampled by guide when computing the ELBO)
            z = pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))
            # decode the latent code z
            loc_img = self.decoder(z)
            # score against actual images
            print(f'{loc_img=}')
            bdist = dist.Bernoulli(loc_img).to_event(1)
            print(f'{bdist=}')
            pyro.sample("obs", bdist, obs=x.reshape(-1, self.image_dimension))
            # pyro.sample("obs", dist.Normal(loc_img, 0.1).to_event(1), obs=x.reshape(-1, self.image_dimension))

    # define the guide (i.e. variational distribution) q(z|x)
    def guide(self, x):
        # register PyTorch module `encoder` with Pyro
        pyro.module("encoder", self.encoder)
        with pyro.plate("data", x.shape[0]):
            # use the encoder to get the parameters used to define q(z|x)
            z_loc, z_scale = self.encoder(x)
            # sample the latent code z
            pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))

    # define a helper function for reconstructing images
    def reconstruct_img(self, x):
        # encode image x
        z_loc, z_scale = self.encoder(x)
        # sample in latent space
        z = dist.Normal(z_loc, z_scale).sample()
        # decode the image (note we don't sample in image space)
        loc_img = self.decoder(z)
        return loc_img

In [39]:
from pyro.optim import ClippedAdam

vae = VAE()
optimizer = ClippedAdam({"lr": 1.0e-3})
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

def train(svi, train_loader, use_cuda=False):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x, _ in train_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
        # do ELBO gradient and accumulate loss
        epoch_loss += svi.step(x)

    # return epoch loss
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = epoch_loss / normalizer_train
    return total_epoch_loss_train

In [40]:
vae.to(device)

VAE(
  (encoder): Encoder(
    (fc1): Linear(in_features=150528, out_features=400, bias=True)
    (fc21): Linear(in_features=400, out_features=50, bias=True)
    (fc22): Linear(in_features=400, out_features=50, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
  )
  (decoder): Decoder(
    (fc1): Linear(in_features=50, out_features=400, bias=True)
    (fc21): Linear(in_features=400, out_features=150528, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
    (sigmoid): Sigmoid()
  )
)

In [41]:
train_elbo = []
for epoch in range(2):
    total_epoch_loss_train = train(svi, dataset.train_dataloader, use_cuda=True)
    train_elbo.append(-total_epoch_loss_train)
    print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))

loc_img=tensor([[0.3643, 0.4692, 0.4283,  ..., 0.3545, 0.6339, 0.5498],
        [0.5261, 0.4305, 0.5122,  ..., 0.3083, 0.5748, 0.6241],
        [0.5419, 0.4872, 0.4511,  ..., 0.4710, 0.5999, 0.5004],
        ...,
        [0.4968, 0.4398, 0.5012,  ..., 0.4959, 0.5198, 0.5171],
        [0.4801, 0.3356, 0.4134,  ..., 0.4375, 0.6360, 0.5183],
        [0.4215, 0.3972, 0.3928,  ..., 0.3917, 0.6307, 0.5447]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
bdist=Independent(Bernoulli(probs: torch.Size([12, 150528])), 1)


ValueError: Error while computing log_prob at site 'obs':
Expected value argument (Tensor of shape (12, 150528)) to be within the support (Boolean()) of the distribution Bernoulli(probs: torch.Size([12, 150528])), but found invalid values:
tensor([[ 1.9578,  1.9407,  1.9407,  ..., -1.2641, -1.2816, -1.2816],
        [ 1.8208,  1.8208,  1.8208,  ..., -0.0092,  0.0082,  0.0431],
        [ 1.4098,  1.3927,  1.3927,  ..., -0.7238, -0.7238, -0.7413],
        ...,
        [ 1.0673,  1.0673,  1.0673,  ...,  0.9842,  0.9668,  0.9494],
        [ 1.5468,  1.5468,  1.5468,  ...,  0.9668,  0.9319,  0.8797],
        [ 2.0777,  2.0777,  2.0777,  ...,  1.8905,  1.8905,  1.8905]],
       device='cuda:0')
        Trace Shapes:                     
         Param Sites:                     
 decoder$$$fc1.weight    400     50       
   decoder$$$fc1.bias           400       
decoder$$$fc21.weight 150528    400       
  decoder$$$fc21.bias        150528       
        Sample Sites:                     
 

In [23]:
x, y = next(iter(dataset.train_dataloader))
output = vae.guide(x.to(device))
print(output.shape)

AttributeError: 'NoneType' object has no attribute 'shape'

In [27]:
from pyro.infer import Predictive

x, y = next(iter(dataset.train_dataloader))
predictive = Predictive(vae.model, guide=vae.guide, num_samples=10,
                        return_sites=("obs", "_RETURN"))  # "linear.weight"
samples = predictive(x.to(device))
predicted_classes = samples['_RETURN'].cpu().numpy()
print(predicted_classes)

TypeError: expected Tensor as element 0 in argument 0, but got NoneType

In [28]:
state_dict = pyro.get_param_store()


In [31]:
state_dict['latent']

KeyError: 'latent'